In [28]:
import snowflake.connector
from datetime import datetime, timedelta
import subprocess
import sys
import os
import pandas as pd
from datetime import datetime, timedelta
import boto3
from google.oauth2 import service_account
from googleapiclient.discovery import build
import snowflake.connector
from dotenv import load_dotenv
load_dotenv()

CLIENT = 'OCF'
# Function to install packages
def install_packages():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    except Exception as e:
        print(f"Failed to install packages from requirements.txt: {e}")
        sys.exit(1)
install_packages()

In [38]:
import snowflake.connector
from datetime import datetime, timedelta
import subprocess
import sys
import os
import pandas as pd
from datetime import datetime, timedelta
import boto3
from google.oauth2 import service_account
from googleapiclient.discovery import build
import snowflake.connector
from dotenv import load_dotenv
load_dotenv()

CLIENT = 'OCF'
# Function to install packages
def install_packages():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    except Exception as e:
        print(f"Failed to install packages from requirements.txt: {e}")
        sys.exit(1)
install_packages()
# Constants and credentials
SITE_URL = os.getenv("SITE_URL")
BUCKET = os.getenv("BUCKET")
FOLDER = os.getenv("FOLDER")
SNOWFLAKE_TABLE = os.getenv("SNOWFLAKE_TABLE")
STAGING_TABLE = os.getenv("STAGING_TABLE")
BATCH_SIZE = int(os.getenv("BATCH_SIZE", "100000"))

GOOGLE_CREDENTIALS_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_DATABASE = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA = os.getenv("SNOWFLAKE_SCHEMA")
SNOWFLAKE_ROLE = os.getenv("SNOWFLAKE_ROLE")

# Snowflake connection parameters

# Connect to Snowflake
def connect_snowflake():
    return snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
    )

# SQL template
SQL_TEMPLATE = """
SET ELIGIBLE_DATE = '{eligible_date}'::DATE;
SET REPORT_ID = '{report_id}';

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
    'BY-DAY' AS ID, 
    $ELIGIBLE_DATE AS START_DATE,        -- Today as the start and end date
    $ELIGIBLE_DATE AS END_DATE,          -- Today as the end date
    CURRENT_DATE AS COLLECT_DATE
UNION ALL
SELECT 
    'BY-WEEK' AS ID, 
    DATE_TRUNC('WEEK', $ELIGIBLE_DATE) AS START_DATE,  -- First day of the current week
    DATEADD('DAY', 6, DATE_TRUNC('WEEK', $ELIGIBLE_DATE)) AS END_DATE,  -- Last day of the current week
    CURRENT_DATE AS COLLECT_DATE
UNION ALL
SELECT 
    'BY-MONTH' AS ID, 
    DATE_TRUNC('MONTH', $ELIGIBLE_DATE) AS START_DATE,  -- First day of the current month
    LAST_DAY($ELIGIBLE_DATE, 'MONTH') AS END_DATE,      -- Last day of the current month
    CURRENT_DATE AS COLLECT_DATE;

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILTERED AS 
SELECT * FROM TIMETABLE  
WHERE ID = $REPORT_ID ;



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_CONSOLE_PAGE_DATA AS
SELECT
    $REPORT_ID AS REPORT_ID,  -- Assign the report ID dynamically
    COMPANY || ' || ' || COMPANY_ID || ' || ' || PAGE AS IDENTIFIER,  -- Concatenate fields with ' || ' in between
    'COMPANY-PAGES' AS LEVEL , 
    START_DATE,
    END_DATE,
    SUM(CLICKS) AS CLICKS,
    SUM(IMPRESSIONS) AS IMPRESSIONS
FROM GOOGLE_BIGQUERY.SEARCH_CONSOLE_RAW.PAGE
INNER JOIN TIMETABLE_FILTERED ON 1=1
WHERE COMPANY_ID IS NOT NULL 
  AND START_DT BETWEEN START_DATE AND END_DATE
GROUP BY 1,2,3,4,5;


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_CONSOLE_COMPANY_DATA AS
SELECT
    $REPORT_ID AS REPORT_ID,  -- Assign the report ID dynamically
    COMPANY || ' || ' || COMPANY_ID  AS IDENTIFIER,  -- Concatenate fields with ' || ' in between
    'COMPANY' AS LEVEL , 
    START_DATE,
    END_DATE,
     SUM(CLICKS) AS CLICKS,
    SUM(IMPRESSIONS) AS IMPRESSIONS
FROM GOOGLE_BIGQUERY.SEARCH_CONSOLE_RAW.PAGE
INNER JOIN TIMETABLE_FILTERED ON 1=1
WHERE COMPANY_ID IS NOT NULL 
  AND START_DT BETWEEN START_DATE AND END_DATE
GROUP BY 1,2,3,4,5;

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE_TIME_SERIES_DATA_V 
WHERE REPORT_ID = $REPORT_ID AND START_DATE IN (SELECT DISTINCT START_DATE FROM TEMP_SEARCH_CONSOLE_COMPANY_DATA)

;

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE_TIME_SERIES_DATA_V
SELECT *,CURRENT_DATE AS INSERT_DATE, CURRENT_TIMESTAMP AS INSERT_TIMESTAMP  FROM TEMP_SEARCH_CONSOLE_COMPANY_DATA
UNION
SELECT *,CURRENT_DATE AS INSERT_DATE, CURRENT_TIMESTAMP AS INSERT_TIMESTAMP  FROM TEMP_SEARCH_CONSOLE_PAGE_DATA
;

"""
def execute_day_by_day(start_date, days_back, report_id):
    conn = connect_snowflake()
    cursor = conn.cursor()
    
    try:
        for i in range(days_back):
            eligible_date = (start_date - timedelta(days=i)).strftime('%Y-%m-%d')
            print(f"Processing for ELIGIBLE_DATE: {eligible_date}, REPORT_ID: {report_id}")
            
            # Prepare SQL with the current date
            sql_to_execute = SQL_TEMPLATE.format(eligible_date=eligible_date, report_id=report_id)
            
            # Split SQL statements by semicolon and execute one by one
            sql_statements = sql_to_execute.strip().split(";")
            for statement in sql_statements:
                if statement.strip():  # Skip empty statements
                    cursor.execute(statement)
                    print(f"Executed: {statement[:50]}...")  # Print the first 50 chars of the statement for context
    except Exception as e:
        print(f"Error executing SQL: {e}")
    finally:
        cursor.close()
        conn.close()




In [ ]:
  'BY-DAY'
'BY-WEEK'

    'BY-MONTH'

In [39]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-09-15", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-09-15, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-09-15'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [40]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-09-22", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-09-22, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-09-22'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [41]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-09-30", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-09-30, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-09-30'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [42]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-10-06", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-10-06, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-10-06'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [43]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-10-13", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-10-13, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-10-13'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [44]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-10-20", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-10-20, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-10-20'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [45]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-10-27", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-10-27, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-10-27'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [46]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-11-04", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-WEEK"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-11-04, REPORT_ID: BY-WEEK
Executed: SET ELIGIBLE_DATE = '2024-11-04'::DATE...
Executed: 
SET REPORT_ID = 'BY-WEEK'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [47]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-11-11", "%Y-%m-%d")  # Start date
    DAYS_BACK = 55  # Number of days to go back
    REPORT_ID = "BY-DAY"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-11-11, REPORT_ID: BY-DAY
Executed: SET ELIGIBLE_DATE = '2024-11-11'::DATE...
Executed: 
SET REPORT_ID = 'BY-DAY'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...
Processing for ELIGIBLE_DATE: 2024-11-10, REPORT_ID: BY-DAY
Executed: SET ELIGIBLE_DATE = '2024-11-10'::DATE...
Executed: 
SET REPORT_ID = 'BY-DAY'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PA

In [48]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-09-11", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-MONTH"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-09-11, REPORT_ID: BY-MONTH
Executed: SET ELIGIBLE_DATE = '2024-09-11'::DATE...
Executed: 
SET REPORT_ID = 'BY-MONTH'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [49]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-10-11", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-MONTH"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-10-11, REPORT_ID: BY-MONTH
Executed: SET ELIGIBLE_DATE = '2024-10-11'::DATE...
Executed: 
SET REPORT_ID = 'BY-MONTH'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...


In [50]:
# Run the script
if __name__ == "__main__":
    START_DATE = datetime.strptime("2024-11-11", "%Y-%m-%d")  # Start date
    DAYS_BACK = 1  # Number of days to go back
    REPORT_ID = "BY-MONTH"  # Change to 'BY-DAY', 'BY-WEEK', or 'BY-MONTH'

    execute_day_by_day(START_DATE, DAYS_BACK, REPORT_ID)

Processing for ELIGIBLE_DATE: 2024-11-11, REPORT_ID: BY-MONTH
Executed: SET ELIGIBLE_DATE = '2024-11-11'::DATE...
Executed: 
SET REPORT_ID = 'BY-MONTH'...
Executed: 

CREATE OR REPLACE TABLE TIMETABLE AS
SELECT 
   ...
Executed: 

CREATE OR REPLACE TEMPORARY TABLE TIMETABLE_FILT...
Executed: 



CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_...
Executed: 


CREATE OR REPLACE TEMPORARY TABLE TEMP_SEARCH_C...
Executed: 

DELETE FROM  GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAGE...
Executed: 

INSERT INTO   GOOGLE_BIGQUERY.SEARCH_CONSOLE.PAG...
